# Train Models

#### Import the required packages and establish the necessary paths

In [1]:
import os, sys, logging, pickle, random, subprocess, joblib
import multiprocessing
import numpy as np
import pandas as pd
import pubchempy as pcp

# Set up base directory paths
BASE_DIR = os.path.curdir
DATA_DIR = os.path.join(BASE_DIR, "CombDrugModule/Data/")

# Add the base directory to the system path to import the custom module
sys.path.append(BASE_DIR)
import CombDrugModule


05-07 10:52:46 INFO     Enabling RDKit 2021.03.5 jupyter extensions


### Attention! Uncomment the following cells only if you need to retrain the models from scratch.

#### Function to return the best parameters for the given score and regressor combination

In [8]:
# # Function to return the best parameters for the given score and regressor combination
# def getBestParams(score_name, reg_name):
#     para_feat_name = "{}_{}".format(score_name, reg_name)
#     ## Define and return parameter sets based on the score and regressor names
#     if para_feat_name == "ZIP_XGBoost":
#         return {'colsample_bylevel': 1.0, 'max_depth': 6, 'n_estimators': 200, 'subsample': 1.0}
#     elif para_feat_name == "Bliss_XGBoost":
#         return {'colsample_bylevel': 1.0, 'max_depth': 6, 'n_estimators': 200, 'subsample': 1.0}
#     elif para_feat_name == "HSA_RandomForest":
#         return {'max_features': 0.3, 'n_estimators': 300}
#     elif para_feat_name == "Loewe_RandomForest":
#         return {'max_features': 0.3, 'n_estimators': 300}

#### Function to train and save the model based on the type of feature and model combination

In [3]:
# def run2SaveTrainModel(type_feat_name):
#     # Extract score and regressor names from the combined feature string
#     score_name, reg_name = type_feat_name.split("_")

#     logging.info("Start {} {}...".format(score_name, reg_name))

#     ## Load feature columns from a pre-saved file
#     with open(os.path.join(DATA_DIR, "{}_{}_feat_name".format(score_name, reg_name)), 'rb') as f:
#         feat_cols = pickle.load(f)

#     ## Load training data from a compressed pickle file
#     logging.info("Load train data...")
#     train_data = pd.read_pickle(os.path.join(DATA_DIR, "train.pickle"),
#                                 compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

#     ## Validate that all required feature columns are present in the training data
#     if len(set(feat_cols).difference([x for x in train_data.columns if x.startswith("feature_")])) > 0:
#         raise ValueError("There is an error in {}".format("type_feat_name"))

#     ## Filter training data to include only the relevant features and target score column
#     train_data = train_data[feat_cols + ["score_{}".format(score_name)]]

#     ## Extract features and target variables for training
#     X_train, y_train = CombDrugModule.getXy(train_data, feature_prefix='feature_',
#                                             y_col_name="score_{}".format(score_name))
#     del train_data

#     ## Train the regressor model
#     logging.info('Train {} start...'.format(reg_name))
#     # Retrieve the best hyperparameters for the current score and regressor combination
#     regressor_param = getBestParams(score_name, reg_name)
#     # Initialize the regressor model
#     model = CombDrugModule.getRegressor(reg_name,
#                                         regressor_params=regressor_param)
#     ## Fit the model to the training data
#     logging.info('Train...')
#     model.fit(X_train, y_train)

#     ## Save the trained model to disk
#     logging.info('Save {} model...'.format(reg_name))
#     joblib.dump(model, os.path.join(DATA_DIR, "{}_{}_train_model".format(score_name, reg_name)))

#     logging.info("Done {} {}...".format(score_name, reg_name))

#### Save the trained models with the optimized sets of hyperparameters

In [4]:
# # List of feature and model combinations to process
# for item in ["ZIP_XGBoost", "Bliss_XGBoost", "HSA_RandomForest", "Loewe_RandomForest"]:
#     run2SaveTrainModel(item)

# # Log that all models have been trained and saved
# logging.info("ALL DONE")

#

# Predict the synergy based on the pairs of drugs' names and the cell's name

In [2]:
# Set an environment variable to limit OpenBLAS threads
os.environ['OPENBLAS_NUM_THREADS'] = '2'
# Set the pre-trained model directory
preTrain_path = DATA_DIR

#### Function to retrieve the SMILES representation of a given drug

In [3]:
def getDrugSmile(drug_name):
    # Query PubChem for the compound based on drug name
    c = pcp.get_compounds(drug_name, "name")
    if len(c) > 0:
        com_flag = c[0]
    else:
        # Try querying by removing any parentheses and extra text
        item = drug_name.split(" (")[0]
        c = pcp.get_compounds(item, "name")
        if len(c) == 0:
            # Raise an error if no compound is found
            raise ValueError("We cannot find {} pubmed cid and corresponding SMILES!".format(drug_name))
        else:
            com_flag = c[0]
            
    # Return the drug name, PubChem CID, and canonical SMILES
    return [drug_name, str(com_flag.cid), str(com_flag.canonical_smiles)]

#### Function to predict four drug combination scores based on the given input data

In [4]:
def predFourScoreResult(input_data, in_type, pre_num):
    """
    Predict four score results based on the specified score models (ZIP and Bliss).
    Returns:
        DataFrame: Predicted results containing drug pairs and predicted scores.
    """
    
    # Copy the input data for training purposes
    data_for_train = input_data.copy()

    # Adjust column names based on the `in_type` value to standardize the feature names
    if in_type == "Row":
        data_for_train = data_for_train.rename(columns={'Drug_1':'Drug_row', 
                                                        'Drug_2':'Drug_col',
                                                        'Drug_1_cid':'Drug_row_cid', 
                                                        'Drug_2_cid':"Drug_col_cid"})
        logging.info("Concat first result ...")
    elif in_type == "Col":
        data_for_train = data_for_train.rename(columns={'Drug_2':'Drug_row', 
                                                        'Drug_1':'Drug_col',
                                                        'Drug_2_cid':'Drug_row_cid', 
                                                        'Drug_1_cid':"Drug_col_cid"})
        logging.info("Concat second result ...")
    
    # Load feature columns to be used in training from a predefined file
    with open(os.path.join(preTrain_path,
                           "HSA_RandomForest_feat_name"), 'rb') as f:
        feat_cols = pickle.load(f)

    # Concatenate features by calling various feature extraction functions
    logging.info("Concat all features ...")
    for func_name in [CombDrugModule.getInfomaxFeatureNew,
                      CombDrugModule.getMordredFeatureNew,
                      CombDrugModule.getRDKitFeatureNew,
                      CombDrugModule.getgeneRawFeatureNew,
                      CombDrugModule.getMutFeatureNew,
                      CombDrugModule.getCNVFeatureNew]:
        feature_df = func_name(data_for_train, pre_num, features=feat_cols)
        data_for_train = pd.concat([data_for_train, 
                                    feature_df.reindex(data_for_train.index)], 
                                   axis=1, sort=False)
    
    # Release memory used by the loaded feature columns
    del feat_cols
    
    # Begin predictions for each specified scoring model
    for item in ["ZIP_XGBoost", "Bliss_XGBoost"]:
        score_name, reg_name = item.split("_")
        logging.info("{} start...".format(score_name))
        
        # Load the feature columns specific to the scoring model
        with open(os.path.join(preTrain_path,
                               "{}_{}_feat_name".format(score_name, reg_name)), 'rb') as f:
            feat_cols = pickle.load(f)
            
        # Extract raw test data features for the current scoring model
        raw_data = data_for_train[feat_cols].copy()
        
        # Load the scaling model and apply it to the test data
        scale_tool = joblib.load(os.path.join(preTrain_path,
                                              "{}_{}_scale_model".format(score_name, reg_name)))
        
        logging.info("Start impute and Scale test...")
        raw_data.loc[:, feat_cols] = scale_tool.transform(raw_data.loc[:, feat_cols])
        
        # Save the indices for the prediction results
        test_data_index = raw_data.index
        
        # Extract the features for prediction
        X_test  = raw_data.filter(regex="^{}".format('feature_'), axis=1).values
        # logging.info("X shape for {} is {}".format(score_name, X_test.shape))
        
        # Release memory used by the raw data
        del raw_data
        
        logging.info("Load train model...")
        # Load the pre-trained model for the current scoring model
        model = joblib.load(os.path.join(preTrain_path,
                                         "{}_{}_train_model".format(score_name, reg_name)))
        
        # Perform predictions using the loaded model
        logging.info('Test {}...'.format(reg_name))
        y_pred = model.predict(X_test)
        
        # Combine prediction results with the original indices
        pred_list = []
        for x, y in zip(test_data_index, y_pred):
            pred_list.append([x, y])
            
        pred_df = pd.DataFrame(pred_list, columns=["DrugComb_id", "pred_{}".format(score_name)])
        pred_df = pred_df.set_index("DrugComb_id")
        
        # Append the predicted scores to the training data
        data_for_train = pd.concat([data_for_train, pred_df], axis=1, sort=False)
        
        # Release memory used by temporary variables
        del feat_cols, pred_df, pred_list
        
    # Define the columns to be included in the final results
    # columns_ = ["Drug_row", "Drug_col", 
    #                                  "Cell_line_name", 
    #                                  "pred_ZIP", "pred_Bliss", 
    #                                  "pred_HSA", "pred_Loewe"]
    columns_ = ["Drug_row", "Drug_col", 
                "Cell_line_name", 
                "pred_ZIP", "pred_Bliss"]
    
    # Filter the training data to include only the relevant columns
    data_for_train = data_for_train[columns_]
    
    # Save the prediction results to a CSV file
    data_for_train.to_csv(os.path.join(CombDrugModule.TMP_DIR,
                                       "Drug_{}_{}".format(in_type, pre_num)),
                                       sep="\t",  float_format='%.3f', index=0)
    return data_for_train


#### Read the input file name from command-line arguments or use default

In [5]:
# Read the input file name from command-line arguments or use default
input_file = "example_input.csv"
output_file = "output.csv"

# Get input data
input_data = pd.read_csv(input_file)

# Get unique drug list from the input data by combining both columns (Drug_1 and Drug_2)
drug_list = list(set(input_data["Drug_1"].unique()).union(set(input_data["Drug_2"].unique())))

# Initialize dictionaries for mapping drug names to CIDs (Chemical IDs) and CIDs to SMILES (chemical structure representation)
drug_name2cid = {}
drug_cid2smile = {}

# Populate the dictionaries using a helper function `getDrugSmile`
for item in drug_list:
    drug_name_list = getDrugSmile(item)  # Example function to map drug names to SMILES
    drug_name2cid[item] = drug_name_list[1]  # Assuming index 1 contains CID
    drug_cid2smile[drug_name_list[1]] = drug_name_list[2]  # Assuming index 2 contains SMILES
    
# Map Drug_1 and Drug_2 to their respective CIDs
input_data["Drug_1_cid"] = input_data["Drug_1"].map(drug_name2cid)
input_data["Drug_2_cid"] = input_data["Drug_2"].map(drug_name2cid)

# Load a cell line mapping file to map cell line names to their DepMap IDs
with open(os.path.join(CombDrugModule.DATA_DIR, "cell_line_map.pickle"), "rb") as f:
    name_map_d = pickle.load(f)
    
# Map cell line names to DepMap IDs in the input data
input_data["DepMap_ID"] = input_data["Cell_line_name"].map(name_map_d)

# Write the CID to SMILES mapping to a DataFrame
cid_df = pd.DataFrame.from_dict(drug_cid2smile, orient='index', columns=["canonical_smiles"])

# Generate a random file prefix number
pre_num = random.randint(1, 9999)

# Ensure that the generated file number is unique
while os.path.isfile(os.path.join(CombDrugModule.TMP_DIR,
                                  "cid2smiles_{}.pickle".format(pre_num))):
    pre_num = random.randint(1, 9999)

# Save the CID to SMILES mapping to a pickle file
with open(os.path.join(CombDrugModule.TMP_DIR,
                       "cid2smiles_{}.pickle".format(pre_num)), "wb") as f:
    pickle.dump(cid_df, f)

# Run Infomax feature extraction using a shell script
logging.info("Starting get Infomax features...")
cmd = "bash {} {}".format(os.path.join(CombDrugModule.MOD_DIR, "runInfomax.sh"),
                          pre_num)
subprocess.call(cmd, shell=True)

# Run Mordred feature extraction
logging.info("Starting get Mordred features...")
CombDrugModule.runForMordred(pre_num)

# Run RDKit feature extraction
logging.info("Starting get RDKit features...")
CombDrugModule.runForRDKit(pre_num)

logging.info("Drug features Done...")

# Extract gene expression features using DepMap IDs
logging.info("Starting get gene expression features...")
CombDrugModule.runGeneNew(list(input_data["DepMap_ID"].unique()), pre_num)

# Extract mutation and CNV features using DepMap IDs
logging.info("Starting get Mutation and CNV features...")
CombDrugModule.runMutNew(list(input_data["DepMap_ID"].unique()), pre_num)
CombDrugModule.runCNVNew(list(input_data["DepMap_ID"].unique()), pre_num)

# Predict four scores (ZIP, Bliss) based on 'Row' and 'Col' arrangements
data_row = predFourScoreResult(input_data, "Row", pre_num)
data_col = predFourScoreResult(input_data, "Col", pre_num)

# Concatenate predicted scores into a single DataFrame
logging.info("Concat the data...")
# data_cols_ = ["pred_ZIP", "pred_Bliss", "pred_HSA", "pred_Loewe"]
data_cols_ = ["pred_ZIP", "pred_Bliss",]
data_concat = pd.concat([data_row, data_col[data_cols_]], axis=1)

# Write final results to an output file
logging.info("Write result...")
# metrics_ = ["ZIP", "Bliss", "HSA", "Loewe"]
metrics_ = ["ZIP", "Bliss",]
for item in metrics_:
    input_data[item] = data_concat["pred_{}".format(item)].mean(axis=1)
    
input_data.to_csv(output_file, sep=",",  float_format='%.3f', index=0)


05-07 10:53:11 INFO     'PUGREST.NotFound: No CID found that matches the given name'
05-07 10:53:14 INFO     Starting get Infomax features...
Using backend: pytorch
05-07 10:53:16 INFO     Starting get Mordred features...
 25%|██▌       | 1/4 [00:01<00:04,  1.49s/it]

/home/ubuntu/miniconda3/envs/drugCombPro/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


/home/ubuntu/miniconda3/envs/drugCombPro/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


05-07 10:53:21 INFO     Starting get RDKit features...
05-07 10:53:22 INFO     Drug features Done...
05-07 10:53:22 INFO     Starting get gene expression features...
05-07 10:53:22 INFO     Starting get Mutation and CNV features...
05-07 10:53:22 INFO     Concat first result ...
05-07 10:53:22 INFO     Concat all features ...
05-07 10:53:22 INFO     ZIP start...
/home/ubuntu/miniconda3/envs/drugCombPro/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/miniconda3/envs/drugCombPro/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.0.2. This might lead to breaking c

In [9]:
df = pd.read_csv('output.csv')
# Convert DataFrame to Markdown-like table
header = '| ' + ' | '.join(df.columns) + ' |'
separator = '| ' + ' | '.join(['---'] * len(df.columns)) + ' |'
rows = '\n'.join(['| ' + ' | '.join(map(str, row)) + ' |' for row in df.values])

# Concatenate everything
table = f"{header}\n{separator}\n{rows}"

# Display using Markdown in a cell
from IPython.display import Markdown, display
display(Markdown(table))

| Drug_1 | Drug_2 | Cell_line_name | Drug_1_cid | Drug_2_cid | DepMap_ID | ZIP | Bliss |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Lapatinib Ditosylate (Tykerb) | Pazopanib hydrochloride | SUM-102PT | 9941095 | 11525740 | ACH-001388 | 32.899 | 60.42 |
| Venetoclax (ABT-199) | Vincristine | SUM-102PT | 49846579 | 5978 | ACH-001388 | 31.121 | 30.944 |
| Lapatinib Ditosylate (Tykerb) | Pazopanib hydrochloride | HCC1395 | 9941095 | 11525740 | ACH-000699 | 4.687 | 0.704 |
| Venetoclax (ABT-199) | Vincristine | HCC1395 | 49846579 | 5978 | ACH-000699 | 9.282 | 5.284 |
| Lapatinib Ditosylate (Tykerb) | Pazopanib hydrochloride | UACC-893 | 9941095 | 11525740 | ACH-000554 | 13.59 | 30.787 |
| Venetoclax (ABT-199) | Vincristine | UACC-893 | 49846579 | 5978 | ACH-000554 | 7.393 | 10.184 |
| Lapatinib Ditosylate (Tykerb) | Pazopanib hydrochloride | Du4475 | 9941095 | 11525740 | ACH-000258 | 8.056 | 11.08 |
| Venetoclax (ABT-199) | Vincristine | Du4475 | 49846579 | 5978 | ACH-000258 | 5.27 | 4.266 |
| Lapatinib Ditosylate (Tykerb) | Pazopanib hydrochloride | HCC1143 | 9941095 | 11525740 | ACH-000374 | -2.979 | 12.168 |
| Venetoclax (ABT-199) | Vincristine | HCC1143 | 49846579 | 5978 | ACH-000374 | -2.481 | -0.592 |